# 조선왕조실록 크롤링 코드

### 조선왕조실록 사이트의 경우 조선왕조실록까지 접근하는데에 있어 많은 단계를 거쳐서 url을 수집하여야 함 

* 맨 처음 작업하는 부분은 사이트에 접속했을 때 조선시대 왕별로 존재하는 특정 key 값을 가져오는 것임 
* 추후 이 값을 활용하여 세부적인 접근을 진행할 예정

In [1]:
import bs4
import requests

In [4]:
tml = requests.get('http://sillok.history.go.kr/search/inspectionList.do').text
dom = bs4.BeautifulSoup(html,'lxml')
def parser(tag):
    return tag.attrs['href'].split('(')[1][1:4]
id_name = list(map(parser,dom.tbody.find_all('a')[::2]))
id_name

['kaa',
 'kba',
 'kca',
 'kda',
 'kea',
 'kfa',
 'kga',
 'kha',
 'kia',
 'kja',
 'kka',
 'kla',
 'kma',
 'kna',
 'knb',
 'koa',
 'kob',
 'kpa',
 'kqa',
 'kra',
 'krb',
 'ksa',
 'ksb',
 'kta',
 'ktb',
 'kua',
 'kva',
 'kwa',
 'kxa',
 'kya']

### 가져온 값 중 한개 활용 request 요청할 때 해당 key값을 보내면 해당 임금님에 대한 사이트로 넘어가짐

In [6]:
html = requests.post(url='http://sillok.history.go.kr/search/inspectionMonthList.do', data={'id':'kaa'}).text
dom = bs4.BeautifulSoup(html,'lxml')
ul = dom.findAll('ul',{'class':'clear2'})[1:-3]
ul = ul[0].findAll('ul')
ul

[<ul class="clear2">
 <li><a href="javascript:search('kaa_101070', '12', '3');">7월</a></li>
 <li><a href="javascript:search('kaa_101080', '12', '3');">8월</a></li>
 <li><a href="javascript:search('kaa_101090', '12', '3');">9월</a></li>
 <li><a href="javascript:search('kaa_101100', '12', '3');">10월</a></li>
 <li><a href="javascript:search('kaa_101110', '12', '3');">11월</a></li>
 <li><a href="javascript:search('kaa_101120', '12', '3');">12월</a></li>
 <li><a href="javascript:search('kaa_101121', '12', '3');">윤12월</a></li>
 </ul>, <ul class="clear2">
 <li><a href="javascript:search('kaa_102010', '21', '3');">1월</a></li>
 <li><a href="javascript:search('kaa_102020', '21', '3');">2월</a></li>
 <li><a href="javascript:search('kaa_102030', '21', '3');">3월</a></li>
 <li><a href="javascript:search('kaa_102040', '21', '3');">4월</a></li>
 <li><a href="javascript:search('kaa_102050', '21', '3');">5월</a></li>
 <li><a href="javascript:search('kaa_102060', '21', '3');">6월</a></li>
 <li><a href="javascrip

### 해당 방법을 활용하여 모든 임금님들에 대한 2번째 key값 얻어오는 작업

In [7]:
id_pass = []
for k in id_name:
    html = requests.post(url='http://sillok.history.go.kr/search/inspectionMonthList.do', data={'id':k}).text
    dom = bs4.BeautifulSoup(html,'lxml')
    ul = dom.findAll('ul',{'class':'clear2'})[1:-3]
    ul = ul[0].findAll('ul')
    for i in ul:
        a = str(i).split('javascript:search(')[1:]
        for j in a:
            id_pass.append(j.split('\'')[1])
id_pass

['kaa_101070',
 'kaa_101080',
 'kaa_101090',
 'kaa_101100',
 'kaa_101110',
 'kaa_101120',
 'kaa_101121',
 'kaa_102010',
 'kaa_102020',
 'kaa_102030',
 'kaa_102040',
 'kaa_102050',
 'kaa_102060',
 'kaa_102070',
 'kaa_102080',
 'kaa_102090',
 'kaa_102100',
 'kaa_102110',
 'kaa_102120',
 'kaa_103010',
 'kaa_103020',
 'kaa_103030',
 'kaa_103040',
 'kaa_103050',
 'kaa_103060',
 'kaa_103070',
 'kaa_103080',
 'kaa_103090',
 'kaa_103100',
 'kaa_103110',
 'kaa_103120',
 'kaa_104010',
 'kaa_104020',
 'kaa_104030',
 'kaa_104040',
 'kaa_104050',
 'kaa_104060',
 'kaa_104070',
 'kaa_104080',
 'kaa_104090',
 'kaa_104091',
 'kaa_104100',
 'kaa_104110',
 'kaa_104120',
 'kaa_105010',
 'kaa_105020',
 'kaa_105030',
 'kaa_105040',
 'kaa_105050',
 'kaa_105060',
 'kaa_105070',
 'kaa_105080',
 'kaa_105090',
 'kaa_105100',
 'kaa_105110',
 'kaa_105120',
 'kaa_106010',
 'kaa_106020',
 'kaa_106030',
 'kaa_106040',
 'kaa_106050',
 'kaa_106060',
 'kaa_106070',
 'kaa_106080',
 'kaa_106090',
 'kaa_106100',
 'kaa_1061

### 2번째 key활용 좀더 깊은 부분의 3번째 key값을 가져오는 작업 실시
* 3번째 key값을 활용하면 해당 임금님의 조선왕조실록에 대한 가장 세부적 접근 가능

In [52]:
url_fin = []
base_url = 'http://sillok.history.go.kr/id/'
for i in id_pass:
    html = requests.post(url='http://sillok.history.go.kr/search/inspectionDayList.do', data={'id':i}).text
    soup = bs4.BeautifulSoup(html,'html.parser')
    a = str(soup.findAll('dl',{'class':'ins_list_main'})[0]).split('searchView(')[1:]
    for j in range(len(a)):
          url_fin.append(base_url + a[j].split("\'")[1])
url_fin

['http://sillok.history.go.kr/id/kaa_10107017_001',
 'http://sillok.history.go.kr/id/kaa_10107017_002',
 'http://sillok.history.go.kr/id/kaa_10107018_001',
 'http://sillok.history.go.kr/id/kaa_10107018_002',
 'http://sillok.history.go.kr/id/kaa_10107018_003',
 'http://sillok.history.go.kr/id/kaa_10107018_004',
 'http://sillok.history.go.kr/id/kaa_10107018_005',
 'http://sillok.history.go.kr/id/kaa_10107020_001',
 'http://sillok.history.go.kr/id/kaa_10107020_002',
 'http://sillok.history.go.kr/id/kaa_10107020_003',
 'http://sillok.history.go.kr/id/kaa_10107026_001',
 'http://sillok.history.go.kr/id/kaa_10107026_002',
 'http://sillok.history.go.kr/id/kaa_10107028_001',
 'http://sillok.history.go.kr/id/kaa_10107028_002',
 'http://sillok.history.go.kr/id/kaa_10107028_003',
 'http://sillok.history.go.kr/id/kaa_10107028_004',
 'http://sillok.history.go.kr/id/kaa_10107028_005',
 'http://sillok.history.go.kr/id/kaa_10107030_001',
 'http://sillok.history.go.kr/id/kaa_10107030_002',
 'http://sil

### 뽑은 URL 저장하기

In [53]:
import pandas as pd

df = pd.DataFrame({'url':url_fin})
df.head()

,url
0,http://sillok.history.go.kr/id/kaa_10107017_001
1,http://sillok.history.go.kr/id/kaa_10107017_002
2,http://sillok.history.go.kr/id/kaa_10107018_001
3,http://sillok.history.go.kr/id/kaa_10107018_002
4,http://sillok.history.go.kr/id/kaa_10107018_003


In [ ]:
df.to_csv('조선왕조실록url.csv',index=False)

### 수집한 URL바탕으로 조선왕조실록 수집

In [1]:
import requests #데이터를 받아오는 역할
from bs4 import BeautifulSoup #데이터에서 값을 뽑아내는 역할
import pandas as pd
import time
import numpy as np
import lxml.html

data = pd.read_csv('조선왕조실록url.csv',engine='python')
length = len(data['url'])

translation = []
original = []
error = []

for i in range(length):
    try:
        req = requests.get(data['url'][i]).text
        root = lxml.html.fromstring(req)
        text = ''
        hanmun = ''
        css = root.cssselect('.paragraph')
        for j in range(len(css)) :
            if j == len(css)-1:
                hanmun += css[j].text_content().strip()
            else:
                text += css[j].text_content().strip()
        translation.append(text)
        original.append(hanmun)
    except:
        print('{}번째에서 오류발생'.format(i))
        error.append(i)
    if i % 10000 == 0 :
        print('-----------------------')
        print('{}번째 진행중'.format(i))

-----------------------
100000번째 진행중
101200번째에서 오류발생
101201번째에서 오류발생
102402번째에서 오류발생
102403번째에서 오류발생
103604번째에서 오류발생
103605번째에서 오류발생
104806번째에서 오류발생
104807번째에서 오류발생
106008번째에서 오류발생
106009번째에서 오류발생
107210번째에서 오류발생
107211번째에서 오류발생
108412번째에서 오류발생
109613번째에서 오류발생
-----------------------
110000번째 진행중
110814번째에서 오류발생
112015번째에서 오류발생
113216번째에서 오류발생
114417번째에서 오류발생
116362번째에서 오류발생
117563번째에서 오류발생
-----------------------
120000번째 진행중
121060번째에서 오류발생
122261번째에서 오류발생
124155번째에서 오류발생
125356번째에서 오류발생
126557번째에서 오류발생
127758번째에서 오류발생
128959번째에서 오류발생
-----------------------
130000번째 진행중
130160번째에서 오류발생
131361번째에서 오류발생
132562번째에서 오류발생
133763번째에서 오류발생
134964번째에서 오류발생
136165번째에서 오류발생
-----------------------
140000번째 진행중
-----------------------
150000번째 진행중
-----------------------
160000번째 진행중
162243번째에서 오류발생
163444번째에서 오류발생
164645번째에서 오류발생
165846번째에서 오류발생
167850번째에서 오류발생
167851번째에서 오류발생
169052번째에서 오류발생
169053번째에서 오류발생
-----------------------
170000번째 진행중
170254번째에서 오류발생
170255번째에서 오류발생
171456번째에서 오류발생


In [4]:
data = pd.DataFrame({'original':original,'translation':translation})
data.to_csv('조선왕조실록fin.csv',index=False,encoding='utf-8')

In [1]:
# 수집 도중 오류가 난 경우 해당 url 저장하기
pd.DataFrame({'error_index':error}).to_csv('에러인댁스.csv',encoding='utf-8',index=False)

NameError: name 'pd' is not defined

# 조선왕조실록 전처리 작업

In [3]:
data = pd.read_csv('조선왕조실록fin.csv',engine='python',encoding='utf-8-sig')
print('총 데이터 수 :',len(data))
data.head()

총 데이터 수 : 348429


,original,translation
0,"○丙申/十七日丙申, 太祖卽位于壽昌宮。 先是, 是月十二日辛卯, 恭讓將幸太祖第, 置酒與...",태조가 수창궁(壽昌宮)에서 왕위에 올랐다. 이보다 먼저 이달 12일에 공양왕(恭讓王...
1,"○上在潛邸, 夢有神人執金尺自天而降, 授之曰: ""慶侍中 復興, 淸矣而已老; 崔都統 瑩...","임금이 잠저(潛邸)에 있을 때, 꿈에 신인(神人)이 금자[金尺]를 가지고 하늘에서 ..."
2,"○丁酉/雨。 前此久旱, 及上卽位, 霈然下雨, 人心大悅。","비가 내리었다. 이보다 앞서 오랫동안 가물었는데, 임금이 왕위에 오르자 억수같이 비..."
3,"○都評議使司及大小臣僚、閑良、耆老等請令知密直司事趙胖赴京申禮部曰: 竊謂小邦, 至恭愍王...",도평의사사(都評議使司) 및 대소 신료(大小臣僚)와 한량(閑良)·기로(耆老) 등이 지...
4,"○立義興親軍衛, 罷都摠中外諸軍事府。",의흥친군위(義興親軍衛)를 설치하고 도총 중외 제군사부(都摠中外諸軍事府)를 폐지하였다.


In [4]:
# 중간중간 수집이 제대로 되지 않은게 있을 수 있기 때문에 dropna 한번 실행 
data = data.dropna(axis=0,how='any')
data.index = range(len(data))
print('총 데이터 수 :',len(data))
data.head()

총 데이터 수 : 348429


,original,translation
0,"○丙申/十七日丙申, 太祖卽位于壽昌宮。 先是, 是月十二日辛卯, 恭讓將幸太祖第, 置酒與...",태조가 수창궁(壽昌宮)에서 왕위에 올랐다. 이보다 먼저 이달 12일에 공양왕(恭讓王...
1,"○上在潛邸, 夢有神人執金尺自天而降, 授之曰: ""慶侍中 復興, 淸矣而已老; 崔都統 瑩...","임금이 잠저(潛邸)에 있을 때, 꿈에 신인(神人)이 금자[金尺]를 가지고 하늘에서 ..."
2,"○丁酉/雨。 前此久旱, 及上卽位, 霈然下雨, 人心大悅。","비가 내리었다. 이보다 앞서 오랫동안 가물었는데, 임금이 왕위에 오르자 억수같이 비..."
3,"○都評議使司及大小臣僚、閑良、耆老等請令知密直司事趙胖赴京申禮部曰: 竊謂小邦, 至恭愍王...",도평의사사(都評議使司) 및 대소 신료(大小臣僚)와 한량(閑良)·기로(耆老) 등이 지...
4,"○立義興親軍衛, 罷都摠中外諸軍事府。",의흥친군위(義興親軍衛)를 설치하고 도총 중외 제군사부(都摠中外諸軍事府)를 폐지하였다.


## 전처리 작업 시행
1. 번역본에 고유명사마다 존재하는 한자제거


2. 각주 및 여백 제거


3. 원문의 길이가 15개이상, 461개이하 개수만 선정 ( 문장의 길이 조절을 위해) 


4. 고유명사에 등장하는 각종 부수적 요소 제거 

In [ ]:
original = []
tranditional = []
for i,j in zip(data['original'],data['translation']):
    j = re.sub(',"',', "',re.sub(r'[0-9]* [ ]*',' ',re.sub(r'[^가-힣,\'\"0-9? .!·]','',j)))
    original.append(i)
    tranditional.append(j)

In [ ]:
df = pd.DataFrame({'original':original,'tranditional':tranditional})
df.to_csv('조선왕조실록fin.csv',encoding='utf-8',index=False)